# Evaluate Model

### Load Packages

In [1]:
import os
import time
import re

### Setup

In [11]:
SETUP_TENSORFLOW = True

WORKSPACE_PATH = "workspace"
DATA_PATH = os.path.join(WORKSPACE_PATH, "data")
PRE_TRAINED_MODELS_PATH = os.path.join(WORKSPACE_PATH, "pre_trained_models")

MODELS_PATH = os.path.join(WORKSPACE_PATH, "models")
BASE_MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711"
MODEL_NAME = "efficientdet_d1_coco17_tpu-32"

PIPELINE_CONFIG_PATH = os.path.join(PRE_TRAINED_MODELS_PATH, MODEL_NAME, "pipeline.config")

NUM_CLASSES = 10

EPOCHS = 50
BATCH_SIZE = 4

OPTIMIZER = "adam" # momentum or adam
LEARNING_RATE = 1e-2 # 1e-2 for momentum, or 1e-3 for adam

if (OPTIMIZER == "adam"):
    LEARNING_RATE = 1e-3
    
USE_AUGMENTATION = "None" # One of "None", "default", or "aligned"
    
    
COMBINATION_NAME = f"epochs_{EPOCHS}-batch_size_{BATCH_SIZE}-optimizer_{OPTIMIZER}-learning_rate_{LEARNING_RATE}-aug_{USE_AUGMENTATION}"

In [57]:
checkpoints = [int(file_name.split('.')[0].split("-")[1]) for file_name in os.listdir(os.path.join(MODELS_PATH, MODEL_NAME, COMBINATION_NAME)) if file_name.startswith("ckpt") and not file_name.endswith("index")]

checkpoints[:10]

TypeError: 'NoneType' object is not subscriptable

In [59]:
#!rm -rf {MODELS_PATH}/{MODEL_NAME}/{COMBINATION_NAME}/eval

In [8]:
COMBINATION_NAME

'epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None'

In [ ]:
start_time = time.time()

!python workspace/model_main_tf2.py \
    --pipeline_config_path=workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/pipeline.config \
    --model_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --checkpoint_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --eval_timeout=0 \
    --alsologtostderr

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Elapsed time: {round(elapsed_time, 2)} minutes")

/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase3GetEPNS_15OpKern

### Download model files
1. Zip all files
2. Download zip

In [10]:
start_time = time.time()
!tar cvfz workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/allfiles.tar.gz workspace/models/{MODEL_NAME}/{COMBINATION_NAME}

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Elapsed time: {round(elapsed_time, 2)} minutes")

workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None/
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None/ckpt-10.data-00000-of-00001
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None/ckpt-90.index
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None/ckpt-45.data-00000-of-00001
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None/ckpt-58.data-00000-of-00001
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None/ckpt-23.index
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-batch_size_4-optimizer_momentum-learning_rate_0.01-aug_None/checkpoint
workspace/models/efficientdet_d1_coco17_tpu-32/epochs_50-

### Export model
Before we can export the model, we have to specify which checkpoint we want to export (per default, the last one is used). This is also used when we want to evaluate on a specific checkpoint.

In [69]:
new_checkpoint = "ckpt-31"

with open(os.path.join(MODELS_PATH, MODEL_NAME, COMBINATION_NAME, 'checkpoint'), 'r+') as file:
    contents = file.read()

    pattern = r'model_checkpoint_path:\s+"(ckpt-\d+)"'
    match = re.search(pattern, contents)

    # Replace the checkpoint number with the new number
    if match:
        old_checkpoint = match.group(1)
        new_contents = contents.replace(old_checkpoint, new_checkpoint, 1)

        # Move the file pointer to the beginning of the file
        file.seek(0)

        # Overwrite the file with the new contents
        file.write(new_contents)

        # Truncate the file to remove any extra characters
        file.truncate()

In [71]:
!python models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path=workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/pipeline.config \
    --trained_checkpoint_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --output_directory=workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/output

/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase3GetEPNS_15OpKern